In [1]:
!pip install plotly
!pip install pandas
!pip install dash

In [24]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px

In [25]:
df= pd.read_csv("ICBC_reported_crashes_Full_Data_data.csv")

In [26]:
flag_symbols = {
    'Animal': 'A',
    'Cyclist': 'C',
    'Motorcycle': 'M',
    'Pedestrian': 'P',
    'Vehicle Only': 'V'
}

flag_columns = list(flag_symbols.keys())[:-1]

def combine_flags(row):
    values = []
    for flag in flag_columns:
        if row[flag + ' Flag'] == 'Y':
            values.append(flag)
    
    if len(values) == 0:
        return 'Vehicle Only'
    else:
        return ', '.join(values)
 

def flags_to_symbols(row):
    flags = row['flags']
    if flags in flag_symbols:
        return flag_symbols[flags]
    else:
        symbols = [flag_symbols[x] for x in flags.split(', ') if x in flag_symbols]
        if len(symbols) == 0:
            return '_'
        else:
            return ''.join(sorted(symbols))

df['flags'] = df.apply(combine_flags, axis=1)
df['flags_symbols'] = df.apply(flags_to_symbols, axis=1)roadOffsets = {"JANUARY": {}, "FEBRUARY":{},"MARCH":{}, "APRIL":{}, "MAY":{}, "JUNE":{}, "JULY":{}, "AUGUST":{}, "SEPTEMBER":{}, "OCTOBER":{}, "NOVEMBER":{}, "DECEMBER":{}}


In [27]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id="graph"),
    html.Button("Go Forward", id='btn_forward', n_clicks=0),
    html.Button("Go Backwards", id='btn_backwards', n_clicks=0)
])

def getx(row):
    return row['table xy'][0]
def gety(row):
    return row['table xy'][1]

def chooseXY(row):
    if row["Street Full Name"] in roadOffsets[row['Month Of Year']]:
        roadOffsets[row['Month Of Year']][row["Street Full Name"]]["x"] += 1
        if roadOffsets[row['Month Of Year']][row["Street Full Name"]]["x"] % 20 == 0:
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["x"] = 0
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] += 0.1
        return roadOffsets[row['Month Of Year']][row["Street Full Name"]]["x"], roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"]
    else:
        roadOffsets[row['Month Of Year']][row["Street Full Name"]] = {}
        roadOffsets[row['Month Of Year']][row["Street Full Name"]]["x"] = 0
        if row['Month Of Year'] == "JANUARY":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 0
        if row['Month Of Year'] == "FEBRUARY":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 1
        if row['Month Of Year'] == "MARCH":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 2
        if row['Month Of Year'] == "APRIL":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 3
        if row['Month Of Year'] == "MAY":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 4
        if row['Month Of Year'] == "JUNE":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 5
        if row['Month Of Year'] == "JULY":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 6
        if row['Month Of Year'] == "AUGUST":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 7
        if row['Month Of Year'] == "SEPTEMBER":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 8
        if row['Month Of Year'] == "OCTOBER":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 9
        if row['Month Of Year'] == "NOVEMBER":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 10
        if row['Month Of Year'] == "DECEMBER":
            roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"] = 11
        return roadOffsets[row['Month Of Year']][row["Street Full Name"]]["x"], roadOffsets[row['Month Of Year']][row["Street Full Name"]]["y"]


df['table xy'] = df.apply (lambda row: chooseXY(row), axis=1)
df['table_x'] = df.apply (lambda row: getx(row), axis=1)
df['table_y'] = df.apply (lambda row: gety(row), axis=1)
    
@app.callback(
    Output("graph", "figure"), 
    [Input("btn_forward", "n_clicks"),
    Input("btn_backwards", "n_clicks")]
    )
def display_graph(n_clicks1, n_clicks2):
    size = 1
    n_clicks = n_clicks1 - n_clicks2
    #only5mask = (df["Street Full Name"] == df["Street Full Name"][0]) | (df["Street Full Name"] == df["Street Full Name"][1]) | (df["Street Full Name"] == df["Street Full Name"][2]) | (df["Street Full Name"] == df["Street Full Name"][3]) | (df["Street Full Name"] == df["Street Full Name"][4])
    RoadNames = []
    top5RoadNames = [0,0,0,0,0]
    for i in range(46519):
        if df["Street Full Name"][i] not in RoadNames:
            RoadNames.append(df["Street Full Name"][i])

    top5RoadNames[0] = RoadNames[n_clicks]
    top5RoadNames[1] = RoadNames[n_clicks + 1]
    top5RoadNames[2] = RoadNames[n_clicks + 2]
    top5RoadNames[3] = RoadNames[n_clicks + 3]
    top5RoadNames[4] = RoadNames[n_clicks + 4]

    

    fig = px.scatter(df[only5mask], x="table_x", y="table_y", opacity=0, text='flags_symbols', facet_col="Street Full Name", height=1000, labels=dict(table_x="", table_y="Month"))
    fig.update_xaxes(showticklabels=False)
    return fig

app.run_server(debug=True, use_reloader=False) 